# YOLOv8 PPE Detection 모델 훈련 (3 Class)

## 개요
이 노트북은 YOLOv8 모델을 PPE 데이터셋(3 class)으로 훈련하는 메인 훈련 스크립트입니다.
Transfer Learning을 활용하여 COCO 사전학습 가중치를 사용합니다.

## 탐지 클래스
- **Class 0**: helmet (헬멧 착용) ✅
- **Class 1**: head (헬멧 미착용) ⚠️
- **Class 2**: vest (안전조끼 착용) ✅

## 실행 과정
1. YOLOv8 모델 로드 (yolov8n.pt - Nano 버전)
2. configs/train_config.yaml에서 하이퍼파라미터 로드
3. configs/ppe_dataset.yaml에서 데이터셋 정보 로드
4. 훈련 실행 및 모델 저장
5. 훈련 결과 및 통계 출력

## 출력 파일
훈련 완료 후 models/ppe_detection/ 폴더에 다음 파일들이 생성됩니다:
- `weights/best.pt`: 최고 성능 모델
- `weights/last.pt`: 마지막 체크포인트
- `results.csv`: 에포크별 훈련 통계
- `confusion_matrix.png`: 혼동 행렬
- `PR_curve.png`: Precision-Recall 곡선
- `results.png`: 훈련 결과 그래프

---

## 1. 라이브러리 임포트

In [ ]:
import os
from pathlib import Path
import yaml
import torch
from dotenv import load_dotenv
from ultralytics import YOLO

print("✅ 라이브러리 임포트 완료")

---

## 2. 헬퍼 함수 정의

In [ ]:
def check_device():
    """
    사용 가능한 디바이스 확인
    
    Returns:
        str: 사용할 디바이스 ('cuda', 'mps', 'cpu')
    """
    if torch.cuda.is_available():
        device = 'cuda'
        device_name = torch.cuda.get_device_name(0)
        print(f"🎮 CUDA GPU 사용 가능: {device_name}")
    elif torch.backends.mps.is_available():
        device = 'mps'
        print("🍎 Apple Silicon MPS 사용 가능")
    else:
        device = 'cpu'
        print("💻 GPU 사용 불가, CPU로 훈련합니다.")
    
    return device


def load_config(config_path):
    """
    훈련 설정 파일 로드
    
    Args:
        config_path: train_config.yaml 파일 경로
    
    Returns:
        dict: 설정 딕셔너리
    """
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    return config

print("✅ 헬퍼 함수 정의 완료")

---

## 3. 프로젝트 경로 설정

In [ ]:
# 프로젝트 루트 경로 설정 (노트북 위치 기준)
# notebooks/training/train_yolov8.ipynb -> 프로젝트 루트
project_root = Path.cwd().parent.parent

# .env 파일 로드
env_path = project_root / '.env'
if env_path.exists():
    load_dotenv(env_path)
    print("✅ .env 파일 로드 완료")
else:
    print("⚠️ .env 파일이 없습니다")

# 경로 확인
print(f"📁 프로젝트 루트: {project_root}")
print(f"📁 현재 디렉토리: {Path.cwd()}")

---

## 4. 훈련 설정

### 4.1 설정 파일 경로 지정

In [ ]:
# 설정 파일 경로
config_path = project_root / 'configs' / 'train_config.yaml'
data_yaml = project_root / 'configs' / 'ppe_dataset.yaml'

print(f"📄 설정 파일: {config_path}")
print(f"📄 데이터셋 파일: {data_yaml}")

# 파일 존재 확인
assert config_path.exists(), f"❌ 설정 파일이 없습니다: {config_path}"
assert data_yaml.exists(), f"❌ 데이터셋 파일이 없습니다: {data_yaml}"
print("✅ 모든 설정 파일 확인 완료")

### 4.2 하이퍼파라미터 설정 (수정 가능)

In [ ]:
# ========================================
# 🔧 이 값들을 수정하여 훈련 설정 변경
# ========================================

# 훈련 기본 설정
EPOCHS = None          # None이면 config 파일 값 사용 (기본: 100)
BATCH_SIZE = None      # None이면 config 파일 값 사용 (기본: 128)
DEVICE = None          # None이면 자동 감지 (cuda, mps, cpu)

# 빠른 테스트용 설정 (MacBook 등)
# EPOCHS = 1
# BATCH_SIZE = 16
# DEVICE = 'mps'

# A100 본 훈련용 설정
# EPOCHS = 100
# BATCH_SIZE = 128
# DEVICE = '0'

### 4.3 설정 파일 로드 및 설정값 추출

In [ ]:
print("="*60)
print("YOLOv8 PPE Detection 모델 훈련 (3 Class)")
print("="*60)
print()

# 설정 파일 로드
print("📋 설정 파일 로드 중...")
config = load_config(config_path)

# 설정값 추출
model_name = config['model']['name']
epochs = EPOCHS if EPOCHS is not None else config['train']['epochs']
batch_size = BATCH_SIZE if BATCH_SIZE is not None else config['train']['batch_size']
img_size = config['train']['img_size']
patience = config['train']['patience']
workers = config['train'].get('workers', 8)

# 디바이스 설정
if DEVICE:
    device = DEVICE
elif config.get('device'):
    device = config['device']
else:
    device = check_device()

# Optimizer 설정
optimizer = config['train']['optimizer']
lr0 = config['train']['lr0']
lrf = config['train']['lrf']
momentum = config['train']['momentum']
weight_decay = config['train']['weight_decay']
warmup_epochs = config['train']['warmup_epochs']

# 출력 설정
save_dir = config['output']['save_dir']
name = config['output']['name']
exist_ok = config['output']['exist_ok']

# 증강 설정
augment = config['augment']

# 설정 출력
print(f"   ✅ 모델: {model_name}")
print(f"   ✅ 데이터셋: {data_yaml}")
print(f"   ✅ Epochs: {epochs}")
print(f"   ✅ Batch Size: {batch_size}")
print(f"   ✅ Image Size: {img_size}")
print(f"   ✅ Device: {device}")
print(f"   ✅ Optimizer: {optimizer}")
print(f"   ✅ Learning Rate: {lr0} -> {lrf}")
print()

---

## 5. YOLOv8 모델 로드

In [ ]:
print("🤖 YOLOv8 모델 로드 중...")

# 사전학습 가중치 로드 (Transfer Learning)
# yolov8n.pt: COCO 데이터셋으로 학습된 Nano 버전
model = YOLO(f"{model_name}.pt")

print(f"   ✅ {model_name}.pt 로드 완료")
print()

---

## 6. 모델 훈련 실행

### ⚠️ 주의사항
- 훈련은 시간이 오래 걸립니다 (A100 GPU 100 epochs: 약 54분)
- 중간에 중단하면 `weights/last.pt`에서 재개 가능
- 훈련 중 실시간 로그가 출력됩니다

In [ ]:
print("🚀 훈련 시작...")
print("-" * 60)

# 훈련 실행
results = model.train(
    # 데이터셋 설정
    data=str(data_yaml),
    
    # 기본 훈련 설정
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    patience=patience,
    workers=workers,
    
    # Optimizer 설정
    optimizer=optimizer,
    lr0=lr0,
    lrf=lrf,
    momentum=momentum,
    weight_decay=weight_decay,
    warmup_epochs=warmup_epochs,
    warmup_momentum=config['train']['warmup_momentum'],
    warmup_bias_lr=config['train']['warmup_bias_lr'],
    
    # 증강 설정
    hsv_h=augment['hsv_h'],
    hsv_s=augment['hsv_s'],
    hsv_v=augment['hsv_v'],
    degrees=augment['degrees'],
    translate=augment['translate'],
    scale=augment['scale'],
    shear=augment['shear'],
    flipud=augment['flipud'],
    fliplr=augment['fliplr'],
    mosaic=augment['mosaic'],
    mixup=augment['mixup'],
    
    # 출력 설정
    project=save_dir,
    name=name,
    exist_ok=exist_ok,
    
    # 디바이스 설정
    device=device,
    
    # 로그 설정
    verbose=config['verbose'],
    save_period=config['save_period'],
    
    # 기타 설정
    save=True,          # 체크포인트 저장
    plots=True,         # 훈련 그래프 저장
    val=True,           # 훈련 중 검증 수행
)

print()
print("="*60)
print("🎉 훈련 완료!")
print("="*60)

---

## 7. 훈련 결과 확인

In [ ]:
# 결과 파일 경로
output_path = Path(save_dir) / name
weights_path = output_path / 'weights'

print("\n📂 결과 파일 위치:")
print(f"   📁 {output_path}")
print()
print("📦 생성된 파일:")
print(f"   🏆 최고 성능 모델: {weights_path / 'best.pt'}")
print(f"   💾 마지막 체크포인트: {weights_path / 'last.pt'}")
print(f"   📊 훈련 통계: {output_path / 'results.csv'}")
print(f"   🔢 혼동 행렬: {output_path / 'confusion_matrix.png'}")
print(f"   📈 PR 곡선: {output_path / 'PR_curve.png'}")
print(f"   📉 훈련 그래프: {output_path / 'results.png'}")
print()

### 7.1 최고 성능 지표 출력

In [ ]:
# 최고 성능 출력
if hasattr(results, 'results_dict'):
    print("\n🏆 최고 성능 통계:")
    metrics = results.results_dict
    
    if 'metrics/mAP50(B)' in metrics:
        print(f"   📊 mAP@0.5: {metrics['metrics/mAP50(B)']:.4f} ({metrics['metrics/mAP50(B)']*100:.2f}%)")
    
    if 'metrics/mAP50-95(B)' in metrics:
        print(f"   📊 mAP@0.5:0.95: {metrics['metrics/mAP50-95(B)']:.4f} ({metrics['metrics/mAP50-95(B)']*100:.2f}%)")
    
    if 'metrics/precision(B)' in metrics:
        print(f"   🎯 Precision: {metrics['metrics/precision(B)']:.4f} ({metrics['metrics/precision(B)']*100:.2f}%)")
    
    if 'metrics/recall(B)' in metrics:
        print(f"   🔍 Recall: {metrics['metrics/recall(B)']:.4f} ({metrics['metrics/recall(B)']*100:.2f}%)")
else:
    print("⚠️ 성능 지표를 가져올 수 없습니다.")

print()

### 7.2 훈련 그래프 시각화

In [ ]:
from IPython.display import Image, display

# 훈련 결과 그래프 표시
results_img = output_path / 'results.png'
if results_img.exists():
    print("📊 훈련 결과 그래프:")
    display(Image(filename=str(results_img)))
else:
    print("⚠️ 훈련 결과 그래프를 찾을 수 없습니다.")

### 7.3 혼동 행렬 시각화

In [ ]:
# 혼동 행렬 표시
confusion_matrix_img = output_path / 'confusion_matrix.png'
if confusion_matrix_img.exists():
    print("🔢 혼동 행렬:")
    display(Image(filename=str(confusion_matrix_img)))
else:
    print("⚠️ 혼동 행렬을 찾을 수 없습니다.")

### 7.4 Precision-Recall 곡선 시각화

In [ ]:
# PR 곡선 표시
pr_curve_img = output_path / 'PR_curve.png'
if pr_curve_img.exists():
    print("📈 Precision-Recall 곡선:")
    display(Image(filename=str(pr_curve_img)))
else:
    print("⚠️ PR 곡선을 찾을 수 없습니다.")

---

## 8. 다음 단계

### 8.1 모델 평가
```bash
uv run python src/4_test/evaluate.py
```

### 8.2 추론 테스트
```bash
uv run python src/3_inference/inference.py --source test_image.jpg
```

### 8.3 모델 파일 경로
- 최고 성능 모델: `models/ppe_detection/weights/best.pt`
- 마지막 체크포인트: `models/ppe_detection/weights/last.pt`

In [ ]:
print("\n" + "="*60)
print("✅ 모든 작업 완료!")
print("="*60)
print("\n다음 단계:")
print("   1️⃣ 모델 평가: uv run python src/4_test/evaluate.py")
print("   2️⃣ 추론 테스트: uv run python src/3_inference/inference.py --source test_image.jpg")
print("   3️⃣ 결과 분석: training_report.md 참고")
print()